<a href="https://colab.research.google.com/github/senthilchandrasegaran/drs2021-tool-share/blob/master/topic_modeling_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyLDAvis
!git clone https://github.com/senthilchandrasegaran/drs2021-tool-share.git

fatal: destination path 'drs2021-tool-share' already exists and is not an empty directory.


In [2]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import gensim
from gensim import corpora, models, utils
from gensim.models import word2vec
from gensim.models import Phrases
from sklearn.manifold import MDS
import numpy as np
from scipy import stats

# import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker

# NLP libraries
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import spacy

import json
import string
import sys
import re
import glob
import pandas as pd
import seaborn as sns

from collections import Counter

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

from IPython.display import Markdown, display

import pprint
pp = pprint.PrettyPrinter(indent=4, width=300)


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packa

In [3]:
files = glob.glob('/content/drs2021-tool-share/song_exploder_transcripts/txts/*.txt')

df_list = []

for filename in files:
    df_file = pd.read_csv(filename, sep=';')
    df_list.append(df_file)

df = pd.concat(df_list, axis=0, ignore_index=True)
print("Read", len(files), "files and loaded", df.shape[0], "rows into the dataframe.")



Read 63 files and loaded 2297 rows into the dataframe.


In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
bigram = Phrases()
sentences = []
lem = WordNetLemmatizer()
tokenizer = TweetTokenizer()
puncts = [ '"', "'", '/', '-', '&', '”', '“']

spoken_stop_words = [
    'ah',
    'em',
    'hm',
    'hmm',
    'like',
    'um',
    'wa',
    'yeah'    
]

complete_stop_words = stopwords.words("english") + spoken_stop_words + puncts

speech_turns = list(df['speech_turn'])

for ind, line in enumerate(speech_turns):
    rawline = line.lower()
    rawline = ''.join([c for c in list(rawline) if c not in string.punctuation])
    tokens = [w for w in tokenizer.tokenize(rawline)]
    lemmas = [lem.lemmatize(token) for token in tokens]
    sentences.append(lemmas)
    bigram.add_vocab([lemmas])
    
uni_bi_gram_tokens = list(bigram[sentences])

print("Uni- and bigram tokens from the first 2 lines:")
print("---------------------------")
for t in uni_bi_gram_tokens[:2]:
    print(t)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Uni- and bigram tokens from the first 2 lines:
---------------------------
['you', '’_re', 'listening', 'to', 'song_exploder', 'where_musician', 'take_apart', 'their_song', 'and', 'piece_by', 'piece_tell', 'the', 'story', 'of', 'how_they', 'were_made', 'my_name', 'is_hrishikesh', 'hirway', '“', 'water_me', 'down', '”_by', 'vagabon']
['when', 'laetitia', 'tamko', 'started', 'making', 'the', 'second', 'vagabon', 'album', 'she', 'really', 'wanted_to', 'produce', 'the', 'entire', 'thing', 'on', 'her_own', 'it', 'would_be', 'a', 'new', 'sound', 'and', 'producing', 'wa', 'still', 'a', 'relatively', 'new', 'skill', 'for', 'her', 'but', 'she', 'wanted_to', 'tackle', 'it', 'head', 'on', 'and', 'do', 'it', 'all', 'herself', 'on', 'this', 'song', 'though', '“', 'water_me', 'down', '”', 'laetitia', 'actually', 'had', 'a', 'coproducer', 'eric', 'littmann', 'it', '’_s', 'the', 'one', 'exception', 'to', 'her', 'otherwise', 'entirely', 'selfproduced', 'album', 'in', 'this_episode', 'she', 'break_down'

# Optional: Identify common n-grams in the corpus

In [5]:
# bigram_counter = Counter()
keys_list = [ x for x in bigram.vocab.keys()]
keylist = []
ngram_tokens = []

for x in bigram.vocab.keys():
    if type(x) == bytes :
        keylist.append(x.decode('UTF-8'))
    else :
        keylist.append(x)
        
# print(keylist)



for ind, key in enumerate(keylist):
    if len(key.split("_")) > 1 :
        # the key is a bigram
        subkey_is_stopword = False
        for subkey in key.split("_") :
            # if either of the words constituting the bigram is a stop word, remove the bigram
            if subkey in complete_stop_words :
                subkey_is_stopword = True
        if not subkey_is_stopword :
            ngram_tokens.append(key)
    else :
        # the key is a unigram
        if not key in complete_stop_words :
            ngram_tokens.append(key)
                
print("Number of uni- and bi-grams found: ", len(ngram_tokens))

# Remove non-occurring bigrams from ngram_tokens
all_tokens = []
for sentence_tokens in uni_bi_gram_tokens:
    all_tokens.extend(sentence_tokens)
    for t in sentence_tokens :
        if t == 'landscape_architect' :
            print(sentence_tokens)
    
print(len(all_tokens), "tokens from", len(uni_bi_gram_tokens), "sentences.")
all_unique_tokens = list(set(all_tokens))
print(len(all_unique_tokens), "unique tokens from", len(uni_bi_gram_tokens), "sentences.")

pruned_ngram_tokens = []
for ngram in ngram_tokens :
    if ngram in all_unique_tokens :
        pruned_ngram_tokens.append(ngram)

print(len(pruned_ngram_tokens), "final uni- and bi-grams found in corpus.")

lem = WordNetLemmatizer()
final_tokens_by_line = []
line_tokens_list = []
lines_to_remove = []
for ind, tokenized_sentence in enumerate(uni_bi_gram_tokens) :
    for token in tokenized_sentence :
        if token == 'landscape_architect' or token == 'landscape_architects':
            print(ind, token)
    stopped_tokens = [w for w in tokenized_sentence if w in pruned_ngram_tokens]
    if len(stopped_tokens) > 0 :
        line_tokens_list.append(stopped_tokens)
    else :
        lines_to_remove.append(speech_turns[ind])
        

if len(lines_to_remove) > 0:
    for del_line in lines_to_remove:
        speech_turns.remove(del_line)

unique_tokens = pruned_ngram_tokens

Number of uni- and bi-grams found:  23759
148911 tokens from 2297 sentences.
7867 unique tokens from 2297 sentences.
7454 final uni- and bi-grams found in corpus.


In [6]:
dictionary = corpora.Dictionary(line_tokens_list)
corpus = [dictionary.doc2bow(line) for line in line_tokens_list]
topic_count = 7
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topic_count, id2word=dictionary, passes=200)
print("")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [7]:
gensimvis.prepare(ldamodel, corpus, dictionary, mds='tsne')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:348: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted t

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
6     -29.594002  60.112713       1        1  48.444300
0     -45.996586  11.891685       2        1  16.631403
2      20.368856  70.019325       3        1  11.999828
5      53.928768  31.704309       4        1   7.703979
3     -12.437127 -26.423267       5        1   5.941814
1       3.965971  21.797720       6        1   4.933218
4      37.524284 -16.517805       7        1   4.345458, topic_info=              Term         Freq        Total Category  logprob  loglift
3    song_exploder   200.000000   200.000000  Default  30.0000  30.0000
918          piano   252.000000   252.000000  Default  29.0000  29.0000
258           drum   220.000000   220.000000  Default  28.0000  28.0000
31           sound   327.000000   327.000000  Default  27.0000  27.0000
30            song  1082.000000  1082.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
265           part    12.501019   212.038131   Topic7  -5.3591   0.3051
867       remember    11.339912   131.989827   Topic7  -5.4566   0.6816
31           sound    11.498215   327.730546   Topic7  -5.4427  -0.2140
18            make    10.857591   282.255634   Topic7  -5.5000  -0.1219
40           music    10.194128   251.705931   Topic7  -5.5631  -0.0704

[463 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
2400      2  0.980047  2018
424       2  0.938151  2019
424       6  0.034746  2019
3081      2  0.966861    30
2343      3  0.985612   808
...     ...       ...   ...
398       2  0.381226  ﬁrst
398       3  0.064890  ﬁrst
398       4  0.048667  ﬁrst
7152      2  0.945867  ﬂows
5089      4  0.919332    ﬂy

[774 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 3, 6, 4, 2, 5])